#### 1. 加载预训练模型

In [26]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        # 继承超类所有方法
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            # 输入图像只有1个通道，尺寸为28×28，padding=2保持卷积后图像大小不变，用6个5×5卷积核进行卷积，输出图尺寸为28×28
            nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5, stride = 1, padding = 2),
            # 在通道上进行BN
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # 第一个池化层输出为6×14×14
        )
        self.conv2 = nn.Sequential(
            # 第二个卷积层，输入为6×14×14，无填充，输出为16×10×10
            nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5, stride = 1),
            # 在通道上进行BN
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # 第二个池化层输出为16×5×5
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            # 论文中该层其实不是FC，且没有激活函数，这里简化了
            nn.Dropout(),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features = 120, out_features = 84),
            nn.Dropout(),
            nn.ReLU()
        )
        # 这里没用径向基函数
        self.fc3 = nn.Linear(in_features = 84, out_features = 10)

    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为一的值，所以要把多维度的tensor展平成一维
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

    # 使用num_flat_features函数计算张量x的总特征量. 比如x是4*2*2的张量，那么它的特征总量就是16
    def num_flat_features(self, x):
        # nn.Conv2d允许输入4维的Tensor：n个样本 x n个通道 x 高度 x 宽度
        # 这里为了计算总特征数，我们只取后三个维度各自的特征，所以是[1:]
        size = x.size()[1:] 
        # 初始化特征数为1
        num_features = 1
        # 将各维度特征数相乘。假设：通道数=3，高=10，宽=10
        # 则num_features = 3*10*10 = 300
        for s in size:
            num_features *= s
        return num_features

if __name__ == '__main__':

    # 加载训练好的LeNet模型
    lenet = torch.load(r'C:\Users\HP\Desktop\PythonLearning\model\LeNet_Trained_Model\LeNet.pkl')
    # 将模型迁移到GPU上进行计算
    lenet = lenet.cuda()

#### 2. 定义显示图片的函数

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(img):
    # 逆归一化
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

#### 3. 定义函数将表格数据转为可以直接喂入网络的4维张量

In [ ]:
def row2tensor(data):
    # 初始化结果集
    a = []
    # 遍历整个数据集
    for i in range(len(data)):
        # 先转为array，存放在列表中
        temp_row = data.loc[i].values
        temp_row = temp_row.reshape((28,28))
        a.append(temp_row)
    # 再转为tensor+移至gpu
    tensordata = torch.from_numpy(np.array(a)).float().cuda()
    tensordata = tensordata.reshape(len(data), 1, 28, 28)
    return tensordata

#### 4. 定义函数计算训练集预测正确率

In [ ]:
def calc_acc(img, train_label):
    output = lenet(img)
    # 对每一列进行softmax归一化，输出预测概率值
    p = F.softmax(output, dim=1)
    # 把CUDA tensor类型的数据转换为numpy类型时要先将其转换成cpu float tensor再转为numpy
    p = p.cpu().detach().numpy()
    # 输出预测类别
    pred_cls = np.argmax(p, axis=1)
    print('预测类别：{}'.format(pred_cls))
    # 计算预测正确率
    num = img.shape[0]  # 批次大小
    acc = sum((pred_cls == train_label[:num]) == 1) / num
    print('预测正确率：{}%'.format(acc * 100))
    return pred_cls

#### 5. 定义函数计算测试集预测类别

In [ ]:
def calc_cls(img):
    output = lenet(img)
    # 对每一列进行softmax归一化，输出预测概率值
    p = F.softmax(output, dim=1)
    # 把CUDA tensor类型的数据转换为numpy类型时要先将其转换成cpu float tensor再转为numpy
    p = p.cpu().detach().numpy()
    # 输出预测类别
    return np.argmax(p, axis=1)

#### 6. 读取训练集前64个数据，可视化数字图像并计算预测正确率

In [ ]:
import pandas as pd

In [ ]:
# 读取训练集数据
train_data = pd.read_csv('train.csv')
train_data

In [ ]:
# 提取训练集标签并转为数组，提取训练集图像数据
train_label = np.array(train_data.iloc[:,0])    # 第一列是标签
train_data = train_data.iloc[:,1:]
print('训练集标签：{}'.format(train_label))
print('训练集图像数据尺寸：{}'.format(train_data.shape))

In [ ]:
# 将训练集数据转为4维张量
train_img = row2tensor(train_data)
train_img.shape    # torch.Size([42000, 1, 28, 28])

In [ ]:
# 预测训练集前64个图像的标签
# 可视化显示这64张图片
imshow(torchvision.utils.make_grid(train_img[:64, :, :, :].cpu()))
calc_acc(train_img[:64, :, :, :], train_label)

#### 7. 计算训练集前28000张图像的预测正确率

In [33]:
calc_acc(train_img[:22000, :, :, :], train_label)

预测类别：[1 0 1 ... 1 1 1]
预测正确率：99.86363636363636%


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

#### 8. 计算测试集预测正确率

In [23]:
# 读取测试集数据
test_data = pd.read_csv('test.csv')
# 将原始数据集转为4维张量
test_img = row2tensor(test_data)
# 输出训练集和测试集张量尺寸
test_img.shape

torch.Size([28000, 1, 28, 28])

In [ ]:
# 可视化部分图像
imshow(torchvision.utils.make_grid(test_img[1900:2000, :, :, :].cpu()))

In [24]:
# 计算预测类别
test_pred_cls = calc_cls(test_img)
test_pred_cls

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

#### 9. 输出预测结果至csv文件中

In [25]:
# 输出预测结果
submission = pd.read_csv('sample_submission.csv')
submission.Label = test_pred_cls
# 删除包含Unnamed的列
# submission = submission.loc[:, ~submission.columns.str.contains('Unnamed')]
submission.to_csv('sample_submission.csv', index=False)